In [128]:
from keras.models import Sequential 

In [129]:
from keras.datasets import mnist
# keras中的mnist数据集已经被划分成了60,000个训练集，10,000个测试集的形式，按以下格式调用即可
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# X_train原本是一个60000*28*28的三维向量，将其转换为60000*784的二维向量
X_train = X_train.reshape(60000, 784)
# X_test原本是一个10000*28*28的三维向量，将其转换为10000*784的二维向量
X_test = X_test.reshape(10000, 784)
# 将X_train, X_test的数据格式转为float32存储
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# 归一化
X_train /= 255
X_test /= 255
# 打印出训练集和测试集的信息
#print(X_train.shape[0], 'train samples')
#print(X_test.shape[0], 'test samples')

#将类别向量(从0到nb_classes的整数向量)映射为二值类别矩阵，
#相当于将向量用one-hot重新编码
from keras.utils import np_utils
# 设置类别的个数
nb_classes = 10
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

In [130]:
from keras.layers.advanced_activations import LeakyReLU 
from keras.optimizers import Adagrad
import numpy as np
from keras.callbacks import EarlyStopping

output_size=32; input_size=784;
encoder = Sequential([Dense(output_size, input_dim=input_size), LeakyReLU(alpha=0.3)])
decoder = Sequential([Dense(input_size, input_dim=output_size), LeakyReLU(alpha=0.3)])
ae = Sequential()
ae.add(encoder)
ae.add(Dropout(p=0.5))
ae.add(decoder)

adg = Adagrad(lr=0.01, epsilon=1e-08, decay=0.0)
ae.compile(optimizer=adg, loss='mean_absolute_error', metrics=['cosine_proximity'])

np.random.seed(1234)
early_stopping = EarlyStopping(monitor='val_loss', patience=20)
hist = ae.fit(x=X_train, y=X_train, 
batch_size=1000, nb_epoch=10, 
verbose=1, callbacks=[early_stopping], 
validation_split=0.15, validation_data=None,
shuffle=True,
class_weight=None, sample_weight=None)
print hist.history

Train on 51000 samples, validate on 9000 samples
Epoch 1/10
51000/51000 [==============================] - 2s - loss: 0.1307 - cosine_proximity: -7.3127e-04 - val_loss: 0.1089 - val_cosine_proximity: -9.0649e-04

In [136]:
model = Sequential()
from keras.layers import Dense, Activation, Dropout
model.add(encoder)
model.add(Dense(512, input_shape=(784,)))

#model.add(Activation('sigmoid'))
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(10))
model.add(Activation("softmax"))

#model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

#from keras.optimizers import SGD
#model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True))

#sgd=SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd, class_mode=None, sample_weight_mode=None)

# 导入优化器RMSProp
from keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
sequential_57 (Sequential)       (None, 32)            25120                                        
____________________________________________________________________________________________________
dense_79 (Dense)                 (None, 512)           16896       leakyrelu_23[0][0]               
____________________________________________________________________________________________________
activation_46 (Activation)       (None, 512)           0           dense_79[0][0]                   
____________________________________________________________________________________________________
dropout_20 (Dropout)             (None, 512)           0           activation_46[0][0]              
___________________________________________________________________________________________

In [137]:
# 设置batch的大小
batch_size = 128
# 设置迭代的次数
nb_epoch = 20
history = model.fit(X_train, y_train,
                    batch_size = batch_size,
                    nb_epoch = nb_epoch,
                    verbose = 1,
                    validation_data = (X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 5s - loss: 0.2753 - acc: 0.9164 - val_loss: 0.1646 - val_acc: 0.9501

In [85]:
score = model.evaluate(X_test, y_test, verbose=0)
print(score)

0.067101557321
